### here we use the ALL data set as an example, replace it with your own data set
- update label variable with name of your own entity types
- your own jsonl file should have one json string at each line
- each json string represents a list of dictionary which must have 'text' and 'label' entry

In [ ]:
import json
with open('data/data.jsonl') as f:
    sents = [json.loads(line.strip()) for line in f]
labels = ['Catalyst', 'Characterization', 'Product', 'Reactant', 'Reaction', 'Treatment']

### here we use the ALL data set as an example, replace it with your own data set
- adjust `lr` and `epoch_number` first
- then play with `neg_rate`
- tune the rest parameter if you like


In [ ]:
lr = 5e-5
wd = 0
epoch_num = 40
neg_rate = 1.4
hidden = 256
dropout = 0.1
batch_size = 32

- checkpoing, output log, tensorboard(monitor loss) are stored under `checkpoint_dir`
- `precision=16` enable half precision training which could accelerate training on newer Nvidia GPU(>Turing)

In [1]:
import json
import logging
import pandas as pd
import pytorch_lightning as pl
from copy import deepcopy
from model import *
from utils import *


bert_name = 'pretrained/scibert_domain_adaption'
checkpoint_dir = 'checkpoint'
checkpoint_name = 'train_demo'

root = logging.getLogger()
logging.basicConfig(level=logging.INFO, format='%(message)s')
root.addHandler(logging.FileHandler(f'{checkpoint_dir}/{checkpoint_name}.log'))
callbacks = [pl.callbacks.ModelCheckpoint(dirpath=checkpoint_dir, filename=f'{checkpoint_name}_checkpoint')]
logger = [pl.loggers.TensorBoardLogger(checkpoint_dir, name=f'{checkpoint_name}_tfboard')]
trainer = pl.Trainer(
    logger=logger,
    enable_checkpointing=True,
    gpus=1,
    precision=16,
    num_sanity_val_steps=0,
    check_val_every_n_epoch=5,
    gradient_clip_val=5,
    max_epochs=epoch_num,
    callbacks=callbacks
)

# here demostrate the train process using the last 1000 sentence as validation data set and the rest as training data set
model = BERTSpan(bert_name, sents[:-1000], sents[-1000:], [], labels, batch_size,
                 neg_rate, hidden, dropout, lr, wd, epoch_num)        


trainer.fit(model)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Global seed set to 12345
Some weights of the model checkpoint at pretrained/scibert_domain_adaption were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClas

/data/zhangyue/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:56: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(



                  precision    recall  f1-score   support

        Catalyst     0.7102    0.8809    0.7864       512
Characterization     0.5876    0.8261    0.6867        69
         Product     0.8515    0.8864    0.8686       440
        Reactant     0.8156    0.9126    0.8614       412
        Reaction     0.7906    0.9205    0.8506       365
       Treatment     0.6870    0.9091    0.7826       198

       micro avg     0.7656    0.8968    0.8260      1996
       macro avg     0.7404    0.8893    0.8061      1996
    weighted avg     0.7713    0.8968    0.8279      1996



                    precision   recall f1-score  support
Catalyst               0.7948   0.9312   0.8576      512
Characterization       0.6460   0.9352   0.7642       69
Product                0.8716   0.9193   0.8948      440
Reactant               0.8587   0.9517   0.9028      412
Reaction               0.8205   0.9616   0.8855      365
Treatment              0.7107   0.9457   0.8115      198
micro avg      


                  precision    recall  f1-score   support

        Catalyst     0.7550    0.8789    0.8123       512
Characterization     0.6042    0.8406    0.7030        69
         Product     0.8688    0.9182    0.8928       440
        Reactant     0.8235    0.9175    0.8680       412
        Reaction     0.8830    0.9507    0.9156       365
       Treatment     0.7572    0.9293    0.8345       198

       micro avg     0.8086    0.9123    0.8573      1996
       macro avg     0.7820    0.9059    0.8377      1996
    weighted avg     0.8126    0.9123    0.8588      1996



                    precision   recall f1-score  support
Catalyst               0.8334   0.9184   0.8738      512
Characterization       0.6580   0.9313   0.7712       69
Product                0.8922   0.9460   0.9183      440
Reactant               0.8639   0.9575   0.9083      412
Reaction               0.8978   0.9662   0.9307      365
Treatment              0.7807   0.9604   0.8613      198
micro avg      


                  precision    recall  f1-score   support

        Catalyst     0.7684    0.8750    0.8183       512
Characterization     0.6484    0.8551    0.7375        69
         Product     0.8637    0.9364    0.8986       440
        Reactant     0.8330    0.9199    0.8743       412
        Reaction     0.8766    0.9534    0.9134       365
       Treatment     0.7922    0.9242    0.8531       198

       micro avg     0.8187    0.9163    0.8648      1996
       macro avg     0.7970    0.9107    0.8492      1996
    weighted avg     0.8207    0.9163    0.8656      1996



                    precision   recall f1-score  support
Catalyst               0.8399   0.9328   0.8839      512
Characterization       0.6969   0.9313   0.7972       69
Product                0.8793   0.9518   0.9141      440
Reactant               0.8688   0.9531   0.9090      412
Reaction               0.8875   0.9680   0.9260      365
Treatment              0.8126   0.9541   0.8777      198
micro avg      


                  precision    recall  f1-score   support

        Catalyst     0.7807    0.8691    0.8226       512
Characterization     0.6552    0.8261    0.7308        69
         Product     0.8278    0.9068    0.8655       440
        Reactant     0.8186    0.8981    0.8565       412
        Reaction     0.8640    0.9397    0.9003       365
       Treatment     0.7902    0.8939    0.8389       198

       micro avg     0.8097    0.8973    0.8512      1996
       macro avg     0.7894    0.8890    0.8357      1996
    weighted avg     0.8107    0.8973    0.8517      1996



                    precision   recall f1-score  support
Catalyst               0.8505   0.9190   0.8834      512
Characterization       0.7031   0.9120   0.7940       69
Product                0.8507   0.9435   0.8947      440
Reactant               0.8572   0.9440   0.8985      412
Reaction               0.8816   0.9575   0.9180      365
Treatment              0.8156   0.9251   0.8669      198
micro avg      


                  precision    recall  f1-score   support

        Catalyst     0.8197    0.8613    0.8400       512
Characterization     0.7692    0.8696    0.8163        69
         Product     0.8779    0.9318    0.9041       440
        Reactant     0.8710    0.9175    0.8936       412
        Reaction     0.8892    0.9452    0.9163       365
       Treatment     0.8211    0.9040    0.8606       198

       micro avg     0.8540    0.9083    0.8803      1996
       macro avg     0.8414    0.9049    0.8718      1996
    weighted avg     0.8542    0.9083    0.8804      1996



                    precision   recall f1-score  support
Catalyst               0.8873   0.8967   0.8920      512
Characterization       0.8034   0.9120   0.8543       69
Product                0.8942   0.9398   0.9165      440
Reactant               0.9050   0.9513   0.9276      412
Reaction               0.9068   0.9598   0.9325      365
Treatment              0.8438   0.9366   0.8878      198
micro avg      


                  precision    recall  f1-score   support

        Catalyst     0.8080    0.8633    0.8347       512
Characterization     0.7403    0.8261    0.7808        69
         Product     0.8545    0.9341    0.8925       440
        Reactant     0.8886    0.9102    0.8993       412
        Reaction     0.8956    0.9397    0.9171       365
       Treatment     0.8203    0.8990    0.8578       198

       micro avg     0.8491    0.9048    0.8761      1996
       macro avg     0.8345    0.8954    0.8637      1996
    weighted avg     0.8498    0.9048    0.8763      1996



                    precision   recall f1-score  support
Catalyst               0.8904   0.9039   0.8971      512
Characterization       0.7944   0.9120   0.8491       69
Product                0.8775   0.9532   0.9138      440
Reactant               0.9232   0.9416   0.9323      412
Reaction               0.9121   0.9521   0.9316      365
Treatment              0.8431   0.9339   0.8862      198
micro avg      


                  precision    recall  f1-score   support

        Catalyst     0.8311    0.8359    0.8335       512
Characterization     0.7568    0.8116    0.7832        69
         Product     0.8487    0.9182    0.8821       440
        Reactant     0.8767    0.9150    0.8955       412
        Reaction     0.9129    0.9479    0.9301       365
       Treatment     0.8333    0.8838    0.8578       198

       micro avg     0.8570    0.8948    0.8755      1996
       macro avg     0.8433    0.8854    0.8637      1996
    weighted avg     0.8570    0.8948    0.8753      1996



                    precision   recall f1-score  support
Catalyst               0.9042   0.8766   0.8902      512
Characterization       0.8131   0.8975   0.8532       69
Product                0.8715   0.9384   0.9037      440
Reactant               0.9114   0.9464   0.9286      412
Reaction               0.9244   0.9571   0.9404      365
Treatment              0.8557   0.9137   0.8838      198
micro avg      


                  precision    recall  f1-score   support

        Catalyst     0.8413    0.8281    0.8346       512
Characterization     0.7671    0.8116    0.7887        69
         Product     0.8614    0.9182    0.8889       440
        Reactant     0.8842    0.9078    0.8958       412
        Reaction     0.9241    0.9342    0.9292       365
       Treatment     0.8294    0.8838    0.8557       198

       micro avg     0.8658    0.8888    0.8771      1996
       macro avg     0.8512    0.8806    0.8655      1996
    weighted avg     0.8660    0.8888    0.8770      1996



                    precision   recall f1-score  support
Catalyst               0.9150   0.8705   0.8922      512
Characterization       0.8139   0.8830   0.8471       69
Product                0.8802   0.9356   0.9071      440
Reactant               0.9193   0.9392   0.9291      412
Reaction               0.9313   0.9388   0.9351      365
Treatment              0.8517   0.9137   0.8816      198
micro avg      